In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb
import numpy as np

# Загрузка данных 
df = pd.read_csv('jamb_exam_results.csv') 

# Преобразование названий колонок к нижнему регистру и замена пробелов на подчеркивания
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Удаление столбца student_id
df = df.drop(columns=['student_id'])

# Заполнение пропущенных значений нулями
df = df.fillna(0)


df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)  # 0.25 * 0.8 = 0.2

# Выделение целевой переменной и признаков
y_train = df_train.jamb_score.values
y_val = df_val.jamb_score.values
y_test = df_test.jamb_score.values

df_train = df_train.drop(columns=['jamb_score'])
df_val = df_val.drop(columns=['jamb_score'])
df_test = df_test.drop(columns=['jamb_score'])

# Преобразование датафреймов в матрицы
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(df_train.to_dict(orient='records'))
X_val = dv.transform(df_val.to_dict(orient='records'))
X_test = dv.transform(df_test.to_dict(orient='records'))

In [11]:
dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train, y_train)

# Получение признака, используемого для разбиения
feature = dv.get_feature_names_out()[dt.tree_.feature[0]]
print("Вопрос 1: Признак для разбиения данных:", feature)

Вопрос 1: Признак для разбиения данных: study_hours_per_week


In [12]:
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_val)
rmse_val = root_mean_squared_error(y_val, y_pred)
print("Вопрос 2: RMSE случайного леса на валидационных данных:", round(rmse_val, 2))

Вопрос 2: RMSE случайного леса на валидационных данных: 42.14


In [13]:
rmse_values = []
for n in range(10, 201, 10):
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    rmse_values.append(rmse)

# Поиск значения n_estimators, после которого RMSE перестает улучшаться
best_n_estimators = 10 + (10 * np.argmin(np.diff(rmse_values) < 0.001))
print("Вопрос 3: n_estimators, после которого RMSE перестает улучшаться:", best_n_estimators)

Вопрос 3: n_estimators, после которого RMSE перестает улучшаться: 90


In [ ]:
best_rmse = float('inf')
best_depth = None
for depth in [10, 15, 20, 25]:
    rmse_list = []
    for n in range(10, 201, 10):
        rf = RandomForestRegressor(n_estimators=n, max_depth=depth, random_state=1, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        rmse_list.append(rmse)
    
    avg_rmse = np.mean(rmse_list)
    if avg_rmse < best_rmse:
        best_rmse = avg_rmse
        best_depth = depth

print("Вопрос 4: Лучшее значение max_depth по среднему RMSE:", best_depth)